## Convolutional Neural Networks

We can create a Convolutional Neural Network to classify dogs and cats using pet images. CNNs are popular for working on image data, and is currently state-of-art for detecting what is contained in an image. The basic CNN structure consists of layers such as Convolution, Pooling and a Fully Connected Layer.

First, we need a dataset and for this we can make use of the [Dogs vs Cats dataset provided by Microsoft](https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765). Within the directory, we have images of cats and dogs and we are looking to convert this dataset into the training set for our model. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

directory = './pet_images'
categories = ['Dog', 'Cat']

training_data = []

def create_training_data():
    for category in categories:
        path = os.path.join(directory, category)
        class_index = categories.index(category)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_arr = cv2.resize(img_arr, (50, 50))
                training_data.append([new_arr, class_index])
            except Exception as e:
                pass

In [2]:
create_training_data()
print(len(training_data))

24946


It's important to ensure the data used for training the model is balanced and there is an equal number of examples for each class (dogs and cats). If this is not done, the model may learn to predict only the class that is the most common and often get stuck there. Fortunately, this Kaggle dataset is already balanced; if it wasn't, we could think about either assigning class weights or trimming the larger set to match size of the smaller set.

Then, it's important to shuffle the data and ensure the samples are not in any particular order. Otherwise, the model will go through all the dog images and learn to always predict dogs before going through the cat images and just predicting cats. Thus, the samples should interspersed amongst each other in order to eliminate bias in our model.

In [3]:
import random

random.shuffle(training_data)

In [4]:
x_train = []
y_train = []

for img, label in training_data:
    x_train.append(img)
    y_train.append(label)

`Convolution` is taking the initial dataset and building feature maps from that. `Pooling` then involves selecting a region and, for example with max-pooling, taking the maximum value of that region, which then becomes the new value for the entire region. `Fully Connected Layers` are typical neural network type of layers (similar to a multilayer perceptron), where all nodes are interconnected. Each `Convolution` and `Pooling` step is a hidden layer.

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

x_train = np.array(x_train).reshape(-1, 50, 50, 1)
y_train = np.array(y_train)

x_train = tf.keras.utils.normalize(x_train, axis=1)

model = Sequential()
model.add(Conv2D(64, (3,3), input_shape = x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    optimizer = 'adam', 
    loss = 'binary_crossentropy', 
    metrics = ['accuracy']
)

model.fit(x_train, y_train, batch_size = 32, validation_split = 0.1, epochs = 5)

Epoch 1/5
702/702 [==============================] - 39s 55ms/step - loss: 0.6282 - accuracy: 0.6465 - val_loss: 0.6086 - val_accuracy: 0.6645
Epoch 2/5
702/702 [==============================] - 42s 60ms/step - loss: 0.5519 - accuracy: 0.7246 - val_loss: 0.5166 - val_accuracy: 0.7655
Epoch 3/5
702/702 [==============================] - 49s 70ms/step - loss: 0.5180 - accuracy: 0.7475 - val_loss: 0.5033 - val_accuracy: 0.7651
Epoch 4/5
702/702 [==============================] - 44s 62ms/step - loss: 0.5008 - accuracy: 0.7604 - val_loss: 0.4894 - val_accuracy: 0.7723
Epoch 5/5
702/702 [==============================] - 42s 60ms/step - loss: 0.4844 - accuracy: 0.7713 - val_loss: 0.4961 - val_accuracy: 0.7655


## Using TensorBoard to analyze the model

TensorBoard is a useful application (which comes with TensorFlow) that allows us to view aspects of our model(s) in browser. We do this via a Keras callback (called `tf.keras.callbacks.TensorBoard`) which enables visualizations for TensorBoard, logging events such as summary plot metrics, training graphs, weight histograms and sampled profiles. 

In [7]:
from tensorflow.keras.callbacks import TensorBoard
import time

model_name = 'pet-cnn-64x2-{}'.format(int(time.time()))
tensorboard = TensorBoard(log_dir = 'logs/{}'.format(model_name))

model.fit(x_train, y_train, batch_size = 32, validation_split = 0.1, epochs = 5, callbacks = [tensorboard])

Epoch 1/5
702/702 [==============================] - 45s 63ms/step - loss: 0.4730 - accuracy: 0.7776 - val_loss: 0.5156 - val_accuracy: 0.7535
Epoch 2/5
702/702 [==============================] - 44s 63ms/step - loss: 0.4597 - accuracy: 0.7830 - val_loss: 0.5113 - val_accuracy: 0.7491
Epoch 3/5
702/702 [==============================] - 42s 60ms/step - loss: 0.4466 - accuracy: 0.7892 - val_loss: 0.4892 - val_accuracy: 0.7671
Epoch 4/5
702/702 [==============================] - 42s 59ms/step - loss: 0.4359 - accuracy: 0.8000 - val_loss: 0.4699 - val_accuracy: 0.7892
Epoch 5/5
702/702 [==============================] - 42s 60ms/step - loss: 0.4259 - accuracy: 0.8051 - val_loss: 0.4716 - val_accuracy: 0.7828
